In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -.5
r1 = 0.5
l2 = -.5
r2 = .5
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np
T_h = 0.2/6
gamma = 5/3
n_fun = lambda x, y, z: np.sin(2*np.pi*x)*np.sin(4*np.pi*y) + 1
#def n_fun(x,y,z):
 #   ind = x < 0.5
  #  out = np.ones_like(x)*2
   # out[x>0.5] = 0.
    #return out
bckgr = GenericCartesianFluidEquilibrium(n_xyz=n_fun)
bckgr.domain = domain

In [ ]:
x = np.linspace(-.5, .5, 100)
y = np.linspace(-.5, .5, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)
eta3 = np.linspace(0,1,1)
ee1, ee2, ee3 = np.meshgrid(eta1, eta2, eta3, indexing="ij")

In [ ]:
import numpy as np
#boxen geben auflösung, die nicht unterschreit bar ist 
from struphy.pic.particles import ParticlesSPH
import matplotlib.pyplot as plt 
nx = 32
ny = 32 
nz = 1
Np = [2**k for k in range(3, 11)]
Np = np.array(Np)
h1 = 1/nx
h2 = 1/ny 
h3 = 1/nz
boxes_per_dim = (nx, ny, nz)
bc = ['periodic']*3
eps = 0.5

error = np.empty((Np.size))  # Fehler-Matrix (hx-Zeilen, Np-Spalten)
n_analytic = n_fun(xx, yy, 0)
shape = np.shape(n_analytic)
print(shape)
n_sq= np.squeeze(n_analytic)
n_sq = n_sq.T
plt.figure(figsize = (12,16))
for i_n, ppb in enumerate(Np):      # Index i_n für Np
    

    particles_Np = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles_Np.draw_markers(sort=False)

    threshold = 0.
    particles_Np.initialize_weights(reject_weights=True, threshold=threshold)

    n_sph_Np = particles_Np.eval_density(
        ee1, ee2, ee3,
        h1=h1, h2=h2, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    
    #A = domain.jacobian_det(eta1,eta2,eta3)
    #print(A)
    n_sph_Np /= domain.jacobian_det(eta1,eta2,eta3)
    n_sph_sq = np.squeeze(n_sph_Np)
    
    plt.subplot(5,2, i_n + 1)
    plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sph_Np[:,:,0])
    plt.axis('square')
    plt.title(f'{ppb=}')
    plt.colorbar()
    
    

    diff = np.max(np.abs(n_sq  - n_sph_sq))
    rel_diff_max = diff/np.max(np.abs(n_sq))
    

    error[i_n] = rel_diff_max  # Speichern des Fehlers
plt.subplot(5,2, 10)
plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sq)
plt.axis('square')
plt.title('n_xyz')
plt.colorbar()
    
    
import matplotlib.pyplot as plt

# Assuming you have already calculated the `Np` and `error` arrays
plt.figure(figsize=(10, 6))
plt.plot(Np, error, label="Error", marker='o', color='b')

plt.xlabel('Number of Particles per Box (Np)', fontsize=14)
plt.ylabel('L2 Norm of Error', fontsize=14)
plt.title('Error vs Np', fontsize=16)
plt.grid(True)
plt.legend()

plt.show()
